# Cannes Lerins H2 - Case study

*Pedro AFFONSO NOBREGA, Yassine ABDELOUADOUD - September 2025*

In 2023 the Cannes Lerins H2 project has been awarded in the "Ecosystèmes Territoriaux" call for projects from the French agency ADEME. The project intends to build a hydrogen refueling station (HRS) with 800 kg/day capacity, in order to support the deployment of 41 hydrogen buses. The HRS will be located in the bus deposit and will include an onsite hydrogen production by electrolysis and a storage capacity of 1 ton of hydrogen. In this suite of case studies we will analyse how this 800 kg/day hydrogen demand could be supplied, with which electricity this hydrogen could be produced and at which cost.

In [4]:
%pip install pommes_craft
%pip install pvlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 124.8 MB/s eta 0:00:00


## Case study 1
In this first case study, we suppose that the onsite electrolyser would be powered exclusively by an unlimited PV above nearby industrial rooftops, as the bus deposit is located in a zone with lots of warehouses. We are interested in determining the required PV surface and in evaluating the cost of a 100% renewable and local electricity supply, in a self consumption mode, without any connection to the electricity grid.

### Data input

We need to set the time horizons for the analysis, both for operation and investment problems. In this section we enter all input data needed to generate the input files for the optimization model. We start by importing some libraries.

In [5]:
import numpy as np
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from io import StringIO
from copy import copy, deepcopy
import copy
from pommes_craft import (
    test_data_path,
    data_path,
    EnergyModel,
    Area,
    Demand,
    EconomicHypothesis,
    ConversionTechnology,
    StorageTechnology,
    Link,
    NetImport,
    TransportTechnology,
    Turpe,
    TimeStepManager,
    Carbon,
    Spillage,
    LoadShedding
)
from pvlib.iotools import read_epw
import logging
import pvlib
from pvlib.pvsystem import PVSystem, Array, FixedMount, retrieve_sam
from pvlib.location import Location
from pvlib.modelchain import ModelChain
import matplotlib.dates as mdates
from datetime import datetime
import matplotlib.ticker as mticker

sns.set_style('ticks')

logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
plt.set_loglevel('WARNING')


#### Basic data

Here we define with some basic data, the years over which we are planning our project and the time steps for investment and operation periods. Here we consider an investment on 2020 ( = year_start - year_inv_step) for operation in 2025. By changing year_end we can change range of the planning period.


In [6]:
year_start = 2025  # First year of the operation period
year_end = 2050  # Last year of the operation period

year_op_step = 10  # Operation period years step
year_inv_step = 10  # Operation period years step

max_lifetime = 25  # Maximal lifetime of equipment (needed to define the decomissioning period range)

year_op = list(range(year_start, year_end + year_op_step, year_op_step))  # List of years where operation will be considered
year_inv = list(range(year_start - year_inv_step, year_end, year_inv_step))  # List of years where investments can be made
year_dec = list(range(year_start, year_end + max_lifetime + year_inv_step, year_inv_step))

hours_step = 6  # Time step for an operation year
hours = list(range(0, 8760, hours_step))

tech_labels = {
    'electrolysis': 'Electrolysis',
    'h2_tank': 'H$_2$ storage',
    'solar_pv': 'Solar PV',
    'h2_trailer': 'H$_2$ trailer'
}
area_colors = {
    "Cannes": "#f4a261",
    "Thorenc": "#2a9d8f",
}

In [7]:
tolerance = 1.e-8

solver_options = {"highs": {},
                  "mosek": {
                        # Optimizer selection for linear problems
                        #     "MSK_IPAR_OPTIMIZER": 0,  # 0=free, 1=interior point, 2=primal simplex, 3=dual simplex
                        # Performance and resource utilization
                        "MSK_IPAR_NUM_THREADS": 20,  # Number of threads (0=automatic)
                        "MSK_DPAR_OPTIMIZER_MAX_TIME": 1000.0,  # Maximum optimization time in seconds
                        "MSK_DPAR_BASIS_REL_TOL_S": tolerance,
                        "MSK_DPAR_INTPNT_TOL_PFEAS": tolerance,
                        "MSK_DPAR_INTPNT_TOL_DFEAS": tolerance,
                      }
                  }


energy_model = EnergyModel(
    name="cannes_lerins_h2",
    hours=hours,  # 24-hour period
    year_ops=year_op,  # Two operation years
    year_invs=year_inv,  # Single investment year
    year_decs=year_dec,  # Single decommissioning year
    modes=[],  # No operation mode
    resources=["electricity", "hydrogen"],  # Basic resources
    hour_types=['base'],
    solver_name="highs",
    solver_options=solver_options['highs']
)

#### Discount and finance rates

Discount and finance rates are important parameters to analyse the economics of capital intensive projects.

The discount rate is used to discount a future amount back to a reference year (here 2020). The finance rate corresponds to the cost of capital to finance capital expenditures, and is used to annualize them. We consider the finance rate equal to the discount rate. We take a discount/finance rate of 4 % which might be considered quite optimistic given the risk associated with hydrogen mobility projects. 4 % is a quite high value for utility PV projects in Europe, as stated in this IEA analysis.

In [8]:
discount_rate = 0.04
finance_rate = 0.04
reference_year = 2020

with energy_model.context():
    economics = EconomicHypothesis("eco", discount_rate=discount_rate, year_ref=reference_year, planning_step=year_inv_step)

#### Time step definitions

We define the time step duration (1h) and operation year duration (8760)

In [9]:
with energy_model.context():
    TimeStepManager("ts_m", time_step_duration=hours_step, operation_year_duration=float(len(hours)))

#### Areas

The study contains one area (node) : Cannes

In [10]:
area_names = ['Cannes',]

with energy_model.context():
    areas = {a_name: Area(a_name) for a_name in area_names}


In [11]:
areas

{'Cannes': <pommes_craft.components.area.Area at 0x7edb1f9f15e0>}

#### Conversion Technologies

This study considers two conversion technologies : solar PV and electrolysis.

For solar PV, we use different investment and fixed costs depending on the region. We obtain the availability of the PV production, i.e. the ratio of the maximum production available to the production capacity, by using the pvlib library with OneClimate epw weather files. PV plants are located at La Roubine bus deposit in Cannes and in Thorenc PV site. PV panels are oriented 160º with respect to the north in Cannes and 180 ºC in Thorenc, and tilted by 35º in both cases.

In [12]:
def import_renewable_ninja_csv(filePath, hours_step, conversion_tech):
    df_pv = pd.read_csv(filePath, header=3)

    df_pv.drop(columns=['time', 'local_time'], inplace=True)
    df_pv.rename(columns={'electricity':conversion_tech}, inplace=True)
    df_pv['hour'] = (df_pv.index / hours_step).astype('int32')
    df_pv = df_pv.groupby('hour').mean()
    return df_pv

df_pv = import_renewable_ninja_csv('./Cannes Lerins H2/data/ninja_pv_la_roubine_2019_tilt_35deg_az_160deg.csv', hours_step, 'solar_pv')
pv_availability = pl.DataFrame({'availability': pl.from_pandas(df_pv), 'hour': pl.Series(hours)})
pv_availability = pl.concat([pv_availability.with_columns(year_op=pl.lit(year_op).cast(pl.Int64)) for year_op in year_op])

FileNotFoundError: [Errno 2] No such file or directory: './Cannes Lerins H2/data/ninja_pv_la_roubine_2019_tilt_35deg_az_160deg.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
interp_years_solar = [2020, 2030, 2040, 2050]

common_pv_params = {
    "factor": {"electricity": 1.0},
    "emission_factor": 0.,
    "variable_cost": 0.,
    "life_span": {year: np.interp(year,
                                  interp_years_solar,
                                  [25, 30, 30, 30])
                  for year in year_inv},
    "finance_rate": finance_rate,
    "power_capacity_investment_max": 1e6,
    "power_capacity_investment_min": 0.,
    "power_capacity_min": 0.,
    "must_run": 1.,

    "fixed_cost": {year: np.interp(year,
                                   interp_years_solar,
                                   [20.e3, 20.e3, 17.e3, 15.e3])
                   for year in year_op},
}
avail_cannes = pv_availability

with energy_model.context():
    solar_pv_cannes = ConversionTechnology(
        name="solar_pv",
        power_capacity_max=1e6,
        availability=avail_cannes,
        invest_cost={year: np.interp(year,
                                     interp_years_solar,
                                     [1067.e3, 867.e3, 757.e3, 597.e3])
                     for year in year_inv},
        **common_pv_params
    )
    areas['Cannes'].add_component(solar_pv_cannes)



Then the electrolysis:

In [ ]:
LHV_H2 = 33.3

electrolysis_capex = {year: 1000 * np.interp(year, [2022,2025,2030,2035,2040,2050],[1596, 1047, 523, 234, 166, 132])
                          for year in year_inv} # EUR/MW
electrolysis_fixed_opex = {year: 0.04 * 1000 * np.interp(year, [2022,2025,2030,2035,2040,2050],[1596, 1047, 523, 234, 166, 132])
                          for year in year_op}   # EUR/MW/yr
electrolysis_variable_opex = {year: 4. * 10 / np.interp(year, [2020,2050],[60, 50])
                          for year in year_op} # EUR/MWh (Water costs at 4 EUR/m3, 10 L/kgH2)
electrolysis_efficiency = {year: LHV_H2/np.interp(year, [2020,2050], [60, 50])
                          for year in year_op} # kWhH2 (LHV)/kWhel
with energy_model.context():
    for area in areas.values():
        electrolysis = ConversionTechnology(
            name="electrolysis",
            factor={
                "electricity": {year: -1. for year in year_op},
                "hydrogen": electrolysis_efficiency,
            },
            emission_factor=0.,
            invest_cost=electrolysis_capex,
            fixed_cost=electrolysis_fixed_opex,
            variable_cost=electrolysis_variable_opex,
            life_span=20.,
            finance_rate=finance_rate,
            power_capacity_investment_max=1000.,
            power_capacity_investment_min=0.,
            power_capacity_max=1.e6,
            power_capacity_min=0.,
            must_run=0.,
        )
        area.add_component(electrolysis)

#### Storage Technologies

Hydrogen storage is provided by H2 tank

In [ ]:
storage_common_params = {
    "factor_in": {"hydrogen": -1.0, 'electricity': 2. / 33.},
    "factor_out": {"hydrogen": 0.8, 'electricity': 0.},
    "factor_keep": {"hydrogen": 0., 'electricity': 0.},
    "life_span": 20.,
    "energy_capacity_investment_min": 0.001,
    "power_capacity_investment_min": 0.,
    "power_capacity_investment_max": 100.,
    'invest_cost_energy': 5400,
    'invest_cost_power': 12600,
    'fixed_cost_energy': 0.,
    'fixed_cost_power': 0.,
    'finance_rate': 0.06,
    'dissipation': 0.,
}



with energy_model.context():
    h2_tank_cannes = StorageTechnology(
        name="h2_tank",
        energy_capacity_investment_max=1e6,
        **storage_common_params
    )
    areas['Cannes'].add_component(h2_tank_cannes)


#### Demand

In this section we define the demand for energy resources. In this case, the only demand is for hydrogen, with 800 kgH2 per day in Cannes. There is no hydrogen demand in Thorenc.

In [ ]:
h2_demand_tons_per_day = 0.800
h2_demand_MW = h2_demand_tons_per_day / 24 * LHV_H2

demand_dict = {
    "electricity": {
        "Cannes": 0.,
    },
    "hydrogen": {
        "Cannes": h2_demand_MW * hours_step,
    }
}

with energy_model.context():
    for area_name, area in areas.items():
        for resource, value in demand_dict.items():
            demand = Demand(
                name=f"{resource}_{area_name}_demand",
                resource=resource,
                demand=value[area_name],
            )
            area.add_component(demand)

            # we allow spillage but not load shedding for electricity and hydrogen

            spillage = Spillage(
                name=f"{resource}_{area_name}_spillage",
                resource=resource,
                max_capacity=1e6,
            )
            area.add_component(spillage)

            load_shedding = LoadShedding(
                name=f"{resource}_{area_name}_load_shedding",
                resource=resource,
                max_capacity=0.,
            )
            area.add_component(load_shedding)

#### Imports
We set the maximum imports and exports possible for each resource to zero, as in this case we consider an autonomous hydrogen production.

In [ ]:
with energy_model.context():
    for area in areas.values():
        net_import = NetImport(
            name="electricity_import",
            resource="electricity",
            import_price=0.,
            max_yearly_energy_import=0.,
            max_yearly_energy_export=0.
        )
        area.add_component(net_import)

        net_import = NetImport(
            name="hydrogen_import",
            resource="hydrogen",
            import_price=0.,
            max_yearly_energy_import=0.,
            max_yearly_energy_export=0.
        )
        area.add_component(net_import)

#### Carbon

We define a Carbon component to measure the total carbon emissions but do not assign a carbon tax or carbon emission limit. A carbon tax is defined here for the example but since no carbon emitting technologies (on a direct emissions scope) are used, it won't affect us.

In [ ]:
with energy_model.context():
    carbon = Carbon("carbon", goal=np.nan, tax=0.05) # EUR/kgCO2eq

### Run model

In [ ]:
case_1 = deepcopy(energy_model)

In [ ]:
case_1.run()

In [ ]:
case_1.set_all_results()

###  Results



In this section we analyse the obtained results and some key indicators.



#### Installed capacities

We start with the installed capacities for the different technologies. We start with electrolysis and solar PV capacities for each area.

In [ ]:
capacities = case_1.get_results('operation', 'power_capacity', [ConversionTechnology])
capacities.pivot(on=['area', 'name'], values='value').with_columns(pl.selectors.all().round(2))

capacities = capacities.rename({
    "year_op": "Year",
    "value": "Installed capacity (MW)",
    "name": "Technology"
})
sns.catplot(
    data=capacities, x="Year", y="Installed capacity (MW)", hue='Technology', col="area",
    kind="bar", height=4, aspect=1.,
)
sns.despine()

We show also the H2 storage capacity in both LHV (MWh H2) and mass (tH2) units.

In [ ]:
storage_capacities = case_1.get_results('operation', 'energy_capacity', [StorageTechnology])
storage_capacities.drop('name').pivot(on=['area'], values='value').with_columns(pl.selectors.all().round(2))
storage_capacities = storage_capacities.with_columns(value=pl.col('value') / 33)

storage_capacities = storage_capacities.rename({
    "year_op": "Year",
    "value": "Installed storage capacity (tH2)",
    "name": "Technology",
    "area": "Area"
})

ax = sns.barplot(data=storage_capacities, x='Year', y='Installed storage capacity (tH2)', hue='Area',
                 palette=area_colors.values())
sns.despine()

#### Load factors
We can also check the load factors for different conversion technologies, as well as the storage level along the year. The load factor is defined as the instant power divided by the power capacity.

In [ ]:
capacities = case_1.get_results('operation', 'power_capacity', [ConversionTechnology]).rename(
    {'value': 'capacity'})
powers = case_1.get_results('operation', 'power', [ConversionTechnology])
avg_power = powers.group_by(['year_op', 'area', 'name']).agg(pl.col('value').mean()).rename({'value': 'avg_power'})
load_factors = (avg_power
                .join(capacities, on=['year_op', 'area', 'name'])
                .with_columns(load_factor=pl.col('avg_power') / pl.col('capacity') * 100.)
                .drop(['avg_power', 'capacity']))
load_factors.pivot(on=['area', 'name'], values='load_factor').with_columns(pl.selectors.all().round(2))

ax = sns.barplot(load_factors.filter(pl.col('name') == 'electrolysis'), x='year_op', y='load_factor', hue='area',
                 palette=area_colors.values())
sns.despine()
ax.set_ylabel('Electrolysis load Factor (%)')
ax.set_xlabel('Year')

#### Hydrogen costs
Finally, we analyse the project costs. We start by the share of each equipment in the (annualized) CAPEX. We can also check the share of different costs in the OPEX.

In [ ]:
capex = case_1.get_results('planning', 'costs').group_by(['year_op', 'name']).agg(pl.col('value').sum())
capex.pivot(on='name', values='value').with_columns(pl.selectors.all().round(0))

capex = capex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=capex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Annual CAPEX (k€)')
ax.set_xlabel('Year')

In [ ]:
opex = case_1.get_results('operation', 'costs').with_columns(
    name=pl.col("name").fill_null('net_import')).group_by(['year_op', 'name']).agg(pl.col('value').sum())
opex.pivot(on=['name'], values='value').with_columns(pl.selectors.all().round(0))

opex = opex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=opex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Annual OPEX (k€)')
ax.set_xlabel('Year')

In [ ]:
totex = (capex
         .join(opex, on=['year_op','name'])
         .with_columns(totex=pl.col('value') + pl.col('value_right'))
         .group_by('year_op').sum())
totex = totex.rename({'value':'capex', 'value_right': 'opex'})

lcoh = economics.discount_rate.with_columns(h2_annual_prod = h2_demand_tons_per_day * 365)
lcoh.with_columns(discounted_annual_prod = pl.col('h2_annual_prod') * (1 - pl.col('discount_rate')) ** (pl.col('year_op') - 2025))

lcoh = lcoh.join(totex, on='year_op')
for value in ['h2_annual_prod','capex','opex','totex']:
  lcoh = lcoh.with_columns((pl.col(value) * (1 - pl.col('discount_rate')) ** (pl.col('year_op') - 2025)).alias('discounted_'+value))
lcoh = lcoh.select('discounted_totex').sum() / lcoh.select('discounted_h2_annual_prod').sum()


print(f'LCOH is {lcoh.item(0,0):.2f} EUR/kgH2')

## Case 2
In this second case study, we suppose that the solar PV capacity will bee limited to both roof and parking surfaces in the bus deposit, corresponding to 4100 m<sup>2</sup> or 800 kWc. However, the electrolyzer can be also run on grid electricity. We are interested in determining the share of grid electricity used to feed the electrolyzer and in evaluating the cost of the hydrogen produced.

In [ ]:
def copy_case(old_case):
  new_case = deepcopy(old_case)
  new_areas = new_case.areas
  for component_index, component in enumerate(energy_model.components):
    for area_name, old_area in old_case.areas.items():
      if component in old_area.components.values():
        new_areas[area_name].components[component.name] = new_case.components[component_index]
  return new_case, new_areas

case_2, areas_case_2 = copy_case(energy_model)

#### Conversion Technologies
PV power capacity is limited to 800 kWc.

In [ ]:
case_2.areas, energy_model.areas

In [ ]:
areas_case_2['Cannes'].components['solar_pv'].power_capacity_max=pl.DataFrame({'year_op':year_op,
                                                                        'power_capacity_max':0.8})


In [ ]:
areas_case_2['Cannes'].components

In [ ]:
energy_model.components

In [ ]:
case_2.components

In [ ]:
case_1.components[3]

In [ ]:
energy_model.components[3]

In [ ]:
case_2.components[3].power_capacity_max

In [ ]:
areas_case_2['Cannes'].components['solar_pv'].power_capacity_max

#### Network costs

We define network costs. Although in France those costs vary in time (find more info on TURPE here), here we consider only average constant costs of 5500 EUR/MW/year (fixed cost) and 18 EUR/MWh (variable cost).
For having only average constant costs, we set the calendar to base for all the time steps of the year.

In [ ]:
with case_2.context():
    turpe = Turpe(
        name='turpe',
        calendar='base',
        fixed_cost=5500.,
        variable_cost=18.
    )

#### Imports
In this case, we are able to use electricity form the grid. We must then set electricity prices at the hourly step at which electricity can be purchased. For this, we use French day-ahead electricity prices obtained from the [entsoe Transparency Platform](https://transparency.entsoe.eu), which contains open data for the European electricity markets. The figure below show those prices between 2019 and 2024. The prices change significantly between the 2020 lows (due to COVID-19 pandemics) and the 2022 energy crisis caused by the Russian invasion of Ukraine. We use 2023 price series as a reference scenario.  

In [ ]:
# Read electricity prices from .csv files
electricity_prices = {
        year: pd.read_csv('./Cannes Lerins H2/data/Day-ahead Prices_entsoe_{:d}.csv'.format(year)) for year in (2019, 2020,2021, 2022,2023,2024)
    }

In [ ]:
# Plot prices time series for different years
fig, ax = plt.subplots(1, 6, figsize=(16,3), sharey=True)
for k, year in enumerate((2019, 2020, 2021, 2022,2023, 2024)):

    ax[k].plot(electricity_prices[year].index, electricity_prices[year]['Day-ahead Price [EUR/MWh]'])
    ax[k].set_title('{}\n{:.0f} EUR/MWh'.format(year,electricity_prices[year]['Day-ahead Price [EUR/MWh]'].mean()))

    ax[k].set_xlabel('Hour')
    ax[k].set_ylim([-100,1000])
    ax[k].set_xlim([0,8760])
ax[0].set_ylabel('French electricity\nday-ahead prices (EUR/MWh)')

In [ ]:
areas_case_2['Cannes'].components['electricity_import'].max_yearly_energy_import=pl.DataFrame({
    'year_op':year_op,
    'max_yearly_energy_import':1e12,
    'area': 'Cannes'
  })
areas_case_2['Cannes'].components['electricity_import'].max_yearly_energy_exmport=pl.DataFrame({
    'year_op':year_op,
    'max_yearly_energy_export':1e12,
    'area': 'Cannes'
  })
areas_case_2['Cannes'].components['electricity_import'].import_price = pl.DataFrame(
    data = {
    'import_price':[hours_step * np.mean(electricity_prices[2023].iloc[hour:hour+hours_step]['Day-ahead Price [EUR/MWh]']) for hour in hours],
    'hour':hours,
    'area': 'Cannes',}
    ).join(other=pl.DataFrame({'year_op':year_op}), how="cross")

In [ ]:
areas_case_2['Cannes'].components['electricity_import'].import_price

In [ ]:
electricity_prices[2023]['Day-ahead Price [EUR/MWh]'].plot()
plt.plot(hours, [np.mean(electricity_prices[2023].iloc[hour:hour+hours_step]['Day-ahead Price [EUR/MWh]']) for hour in hours])
plt.plot(hours, areas_case_2['Cannes'].components['electricity_import'].import_price.filter(pl.col('year_op') == 2035).select('import_price'))

In [ ]:
np.mean([np.mean(electricity_prices[2023].iloc[hour:hour+hours_step]['Day-ahead Price [EUR/MWh]']) for hour in hours]) / 6

In [ ]:
areas_case_2['Cannes'].components['electricity_import'].import_price

In [ ]:
case_3, areas_case_3 = copy_case(case_2)

### Run model

In [ ]:
case_2.run()

In [ ]:
case_2.set_all_results()

###  Results



In this section we analyse the obtained results and some key indicators.



#### Installed capacities

We start with the installed capacities for the different technologies. We start with electrolysis and solar PV capacities for each area.

In [ ]:
capacities = case_2.get_results('operation', 'power_capacity', [ConversionTechnology])
capacities.pivot(on=['area', 'name'], values='value').with_columns(pl.selectors.all().round(2))

capacities = capacities.rename({
    "year_op": "Year",
    "value": "Installed capacity (MW)",
    "name": "Technology"
})
sns.catplot(
    data=capacities, x="Year", y="Installed capacity (MW)", hue='Technology', col="area",
    kind="bar", height=4, aspect=1.,
)
sns.despine()

We show also the H2 storage capacity in both LHV (MWh H2) and mass (tH2) units.

In [ ]:
storage_capacities = case_2.get_results('operation', 'energy_capacity', [StorageTechnology])
storage_capacities.drop('name').pivot(on=['area'], values='value').with_columns(pl.selectors.all().round(2))
storage_capacities = storage_capacities.with_columns(value=pl.col('value') / 33)

storage_capacities = storage_capacities.rename({
    "year_op": "Year",
    "value": "Installed storage capacity (tH2)",
    "name": "Technology",
    "area": "Area"
})

ax = sns.barplot(data=storage_capacities, x='Year', y='Installed storage capacity (tH2)', hue='Area',
                 palette=area_colors.values())
sns.despine()

#### Load factors
We can also check the load factors for different conversion technologies, as well as the storage level along the year. The load factor is defined as the instant power divided by the power capacity.

In [ ]:
capacities = case_2.get_results('operation', 'power_capacity', [ConversionTechnology]).rename(
    {'value': 'capacity'})
powers = case_2.get_results('operation', 'power', [ConversionTechnology])
avg_power = powers.group_by(['year_op', 'area', 'name']).agg(pl.col('value').mean()).rename({'value': 'avg_power'})
load_factors = (avg_power
                .join(capacities, on=['year_op', 'area', 'name'])
                .with_columns(load_factor=pl.col('avg_power') / pl.col('capacity') * 100.)
                .drop(['avg_power', 'capacity']))
load_factors.pivot(on=['area', 'name'], values='load_factor').with_columns(pl.selectors.all().round(2))

ax = sns.barplot(load_factors.filter(pl.col('name') == 'electrolysis'), x='year_op', y='load_factor', hue='area',
                 palette=area_colors.values())
sns.despine()
ax.set_ylabel('Electrolysis load Factor (%)')
ax.set_xlabel('Year')

#### Hydrogen costs
Finally, we analyse the project costs. We start by the share of each equipment in the (annualized) CAPEX. We can also check the share of different costs in the OPEX.

In [ ]:
capex = case_2.get_results('planning', 'costs').group_by(['year_op', 'name']).agg(pl.col('value').sum())
capex.pivot(on='name', values='value').with_columns(pl.selectors.all().round(0))

capex = capex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=capex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Annual CAPEX (k€)')
ax.set_xlabel('Year')

In [ ]:
opex = case_2.get_results('operation', 'costs').with_columns(
    name=pl.col("name").fill_null('net_import')).group_by(['year_op', 'name']).agg(pl.col('value').sum())
opex.pivot(on=['name'], values='value').with_columns(pl.selectors.all().round(0))

opex = opex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=opex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Annual OPEX (k€)')
ax.set_xlabel('Year')

In [ ]:
totex = (capex
         .join(opex, on=['year_op','name'], how='right')
         .group_by('year_op').sum()).with_columns(totex=pl.col('value') + pl.col('value_right'))
totex = totex.rename({'value':'capex', 'value_right': 'opex'})

lcoh = economics.discount_rate.with_columns(h2_annual_prod = h2_demand_tons_per_day * 365)
lcoh.with_columns(discounted_annual_prod = pl.col('h2_annual_prod') * (1 - pl.col('discount_rate')) ** (pl.col('year_op') - 2025))

lcoh = lcoh.join(totex, on='year_op')
for value in ['h2_annual_prod','capex','opex','totex']:
  lcoh = lcoh.with_columns((pl.col(value) * (1 - pl.col('discount_rate')) ** (pl.col('year_op') - 2025)).alias('discounted_'+value))
lcoh = lcoh.select('discounted_totex').sum() / lcoh.select('discounted_h2_annual_prod').sum()


print(f'LCOH is {lcoh.item(0,0):.2f} EUR/kgH2')

In [ ]:
# sol2 = case_2.linopy_model.solution

# # Get the electrolysis electrical power for Cannes area
# electrolysis_power = case_2.get_results('operation', 'power').filter(pl.col' == '')

# # Get the solar PV electrical power for Cannees area
# solar_pv_power = sol2['operation_conversion_power'].loc[{'area': 'Cannes', 'conversion_tech':'solar_pv'}].sum('year_op')

# # Plot as a function of time for each operation year.
# fig, ax = plt.subplots(nrows=len(year_op), ncols=1, figsize=(8,4 * len(year_op)), sharey=True, sharex=True, squeeze=False)

# for k, y in enumerate(year_op):
#     ax[k,0].set_title(y)
#     ax[k,0].plot(hours, solar_pv_power, label='Solar PV')
#     ax[k,0].plot(hours, electrolysis_power, label='Water electrolysis')
# ax[k,0].set_xlabel('Hour')
# ax[k,0].set_ylabel('Power (MW$_{el}$)')
# ax[k,0].legend(loc='upper right')

# # We can set the limits here to zoom into a particular period of the year
# ax[k,0].set_xlim([2000,2400])
# fig.tight_layout()

## Case 3

## Transport Technologies

In this case, we need to set the parameters for the exchange technologies between the two areas, Cannes and Thorenc. We consider a hydrogen tube trailer. The investment cost is calculated from the truck cost, the exchange capacity from the truck capacity and the trip parameters (distance, duration) and the variable cost from both the driver labor cost and fuel costs. We consider a minimum of 1 truck (to avoid unrealistic fractional values lower than 1).

Data for the tube trailer is obtained from Laguipo, Forde & Carton. Int. J. Hydrogen Energy (2022).

In [ ]:
t_trip = 1.5  # h
t_load = 0.75  # h
M_trailer = 0.4  # tH2
P_truck = M_trailer * LHV_h2 / 2 / (t_trip + t_load)  # MWH2
C_truck = 200e3  # EUR
C_labor = 27.8  # EUR/h
C_diesel = 1.7  # EUR/L
diesel_cons_truck = 40. / 100.  # L/km
L_trip = 70.  # km

# Laguipo, J., Forde, C. & Carton, J. G. Enabling the scale up of green hydrogen in Ireland by decarbonising the haulage sector. Int. J. Hydrogen Energy (2022) doi:10.1016/j.ijhydene.2022.06.053.


with energy_model.context():
    transport_technology = TransportTechnology(
        "h2_trailer",
        resource="hydrogen",
        life_span=15.,
        invest_cost=C_truck / P_truck,
        fixed_cost=0.02 * C_truck / P_truck,
        hurdle_costs=C_labor / P_truck + C_diesel * diesel_cons_truck * (2 * L_trip) / (M_trailer * LHV_h2),
        finance_rate=0.06,
        power_capacity_investment_max=10 * P_truck,
        power_capacity_investment_min=1 * P_truck,
    )
    link = Link(
        name="hydrogen_link",
        area_from=areas['Thorenc'],
        area_to=areas['Cannes'],
    )
    link.add_transport_technology(transport_technology)


## Imports

In this case, we are able to use electricity form the grid. We must then set electricity prices at the hourly step at which electricity can be purchased. For this, we use French day-ahead electricity prices obtained from the entsoe Transparency Platform, which contains open data for the European electricity markets. The figure below show those prices between 2014 and 2024. The prices change significantly between the 2020 lows (due to COVID-19 pandemics) and the 2022 energy crisis caused by the Russian invasion of Ukraine. We use 2023 price series as a reference scenario.

In [ ]:
fr_price_path = data_path / 'electricity_prices' / 'France.csv'
prices_fr = pl.read_csv(fr_price_path)
fig, ax = plt.subplots(figsize=(10, 4))
plt.plot(prices_fr["Datetime (Local)"].str.to_datetime(),
         prices_fr["Price (EUR/MWhe)"], linewidth=0.2)

In [ ]:
def get_electricity_prices(price_years, year_ops, hours):
    fr_price_path = data_path / 'electricity_prices' / 'France.csv'
    prices_fr = pl.read_csv(fr_price_path)
    prices_fr = prices_fr.with_columns(time_index=pl.col('Datetime (Local)').str.to_datetime())

    price_by_year = []

    for i, price_year in enumerate(price_years):
        year_op = year_ops[i]
        print(f"Price from year {price_year} used for operation year : {year_op}")
        m = pl.col('time_index').dt.year() == price_year
        m_2902 = ~((pl.col('time_index').dt.month() == 2) & (pl.col('time_index').dt.day() == 29))
        prices_ = prices_fr.filter(m & m_2902)
        prices_ = (prices_
                   .select('Price (EUR/MWhe)')
                   .rename({'Price (EUR/MWhe)': 'import_price'})
                   .with_columns(pl.Series('hour', hours), year_op=pl.lit(year_op).cast(pl.Int64)))
        price_by_year.append(prices_)

    return pl.concat(price_by_year)

We create the polars DataFrame that contains the import prices for each operation year and then we pass it to create the NetImport instance

In [ ]:
import_price = get_electricity_prices([2023] * len(year_op), year_op, hours)

with energy_model.context():
    for area in areas.values():
        net_import = NetImport(
            name="electricity_import",
            resource="electricity",
            import_price=import_price,
            max_yearly_energy_export=0.
        )
        area.add_component(net_import)

        net_import = NetImport(
            name="hydrogen_import",
            resource="hydrogen",
            import_price=0.,
            max_yearly_energy_import=0.,
            max_yearly_energy_export=0.
        )
        area.add_component(net_import)

## Case study 4

In this third case study, we suppose that the solar PV capacity will bee limited to both roof and parking surfaces in the bus deposit, corresponding to 4100 m2 or 800 kWc, and that the electrolyzer can be also run on grid electricity, as in case 2. However, we limit hydrogen storage capacity to 1 ton, to account for regulatory constraints.

## Areas

The study contains two areas : Cannes and Thorenc

In [ ]:
area_names = ['Cannes', 'Thorenc']

with energy_model.context():
    areas = {a_name: Area(a_name) for a_name in area_names}


##  Results

In this section we analyse the obtained results and some key indicators.

### Installed capacities

We start with the installed capacities for the different technologies. We start with electrolysis and solar PV capacities for each area.

In [ ]:
capacities = energy_model.get_results('operation', 'power_capacity', [ConversionTechnology])
capacities.pivot(on=['area', 'name'], values='value').with_columns(pl.selectors.all().round(2))

In [ ]:
capacities = capacities.rename({
    "year_op": "Year",
    "value": "Installed Capacity (MW)",
    "name": "Technology"
})
sns.catplot(
    data=capacities, x="Year", y="Installed Capacity (MW)", hue='Technology', col="area",
    kind="bar", height=4, aspect=1.,
)
sns.despine()

We show also the H2 storage capacity in both LHV (MWh H2) and mass (tH2) units.

In [ ]:
storage_capacities = energy_model.get_results('operation', 'energy_capacity', [StorageTechnology])
storage_capacities.drop('name').pivot(on=['area'], values='value').with_columns(pl.selectors.all().round(2))

In [ ]:
storage_capacities = storage_capacities.rename({
    "year_op": "Year",
    "value": "Installed Capacity (MWh H2)",
    "name": "Technology"
})

ax = sns.barplot(data=storage_capacities, x='Year', y='Installed Capacity (MWh H2)', hue='area',
                 palette=area_colors.values())
sns.despine()

And the tube trailer capacity to transport hydrogen between Thorenc and Cannes.

In [ ]:
transport_capacities = energy_model.get_results('operation', 'power_capacity', [TransportTechnology])
transport_capacities.pivot(on=['name'], values='value').with_columns(
    **{"h2 trailer tH2/day": pl.col("h2_trailer") / LHV_h2 * 24,
       "Number of trucks": pl.col("h2_trailer") / P_truck}).with_columns(pl.selectors.all().round(2))

## Load factors
We can also check the load factors for different conversion technologies, as well as the storage level along the year. The load factor is defined as the instant power divided by the power capacity.

In [ ]:
capacities = energy_model.get_results('operation', 'power_capacity', [ConversionTechnology]).rename(
    {'value': 'capacity'})
powers = energy_model.get_results('operation', 'power', [ConversionTechnology])
avg_power = powers.group_by(['year_op', 'area', 'name']).agg(pl.col('value').mean()).rename({'value': 'avg_power'})
load_factors = (avg_power
                .join(capacities, on=['year_op', 'area', 'name'])
                .with_columns(load_factor=pl.col('avg_power') / pl.col('capacity') * 100.)
                .drop(['avg_power', 'capacity']))
load_factors.pivot(on=['area', 'name'], values='load_factor').with_columns(pl.selectors.all().round(2))

In [ ]:
ax = sns.barplot(load_factors.filter(pl.col('name') == 'electrolysis'), x='year_op', y='load_factor', hue='area',
                 palette=area_colors.values())
sns.despine()
ax.set_ylabel('Load Factor (%)')
ax.set_xlabel('Year')

## Hydrogen costs
Finally, we analyse the project costs. We start by the share of each equipment in the (annualized) CAPEX. We can also check the share of different costs in the OPEX.

In [ ]:
capex = energy_model.get_results('planning', 'costs').group_by(['year_op', 'name']).agg(pl.col('value').sum())
capex.pivot(on='name', values='value').with_columns(pl.selectors.all().round(0))

In [ ]:
capex = capex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=capex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Capex (in k€)')
ax.set_xlabel('Year')

In [ ]:
opex = energy_model.get_results('operation', 'costs').with_columns(
    name=pl.col("name").fill_null('net_import')).group_by(['year_op', 'name']).agg(pl.col('value').sum())
opex.pivot(on=['name'], values='value').with_columns(pl.selectors.all().round(0))

In [ ]:
opex = opex.sort(by=['year_op', 'value']).with_columns(value=pl.col('value') * 1.e-3)
ax = sns.barplot(data=opex, x='year_op', y='value', hue='name', palette="mako")
sns.despine()
ax.legend(bbox_to_anchor=(1, 1), frameon=False)
ax.set_ylabel('Opex (in k€)')
ax.set_xlabel('Year')

We can check the mean price payed for electricity purchased from the grid.

In [ ]:
total_cost = net_import.results['operation']['costs']['value'].sum() + turpe.results['operation']['fixed_costs'][
    'value'].sum() + turpe.results['operation']['variable_costs']['value'].sum()
total_imports = net_import.results['operation']['import']['value'].sum()
mean_electricity_price = total_cost / total_imports
print("Mean imported electricity price : {:.1f} EUR/MWh".format(mean_electricity_price))

We can also check the cost to transport hydrogen from Thorenc to Cannes with tube trailers on EUR/kgH2 basis.

In [ ]:
total_costs = transport_technology.results['planning']['costs']['value'].sum() + \
              transport_technology.results['operation']['costs']['value'].sum()
total_power = transport_technology.results['operation']['power']['value'].sum()
print('H2 transport cost: {:.2f} EUR/kgH2'.format(total_costs / total_power / LHV_h2 * 10))
